In [ ]:
import pandas as pd
import numpy as np

import os
import sys
import cv2

import seaborn as sns
import torch
import requests
import pickle
import joblib

from google.colab.patches import cv2_imshow

from PIL import Image
from io import BytesIO

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import HTML,display, Javascript,HTML, Audio, Image
from google.colab.output import eval_js
import base64
from base64 import b64decode
from moviepy.editor import VideoFileClip

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import keras
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Activation,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from keras.models import Sequential,load_model
from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.utils.np_utils import to_categorical

import tensorflow
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop



In [ ]:
def record_video(filename):
  js=Javascript("""
    async function recordVideo() {
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");

      capture.textContent = "Start Recording";
      capture.style.background = "orange";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});

      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);

      video.srcObject = stream;
      video.muted = true;

      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);

      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();
      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();

      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
    return btoa(binaryString);
    }
  """)
  try:
    display(js)
    data=eval_js('recordVideo({})')
    binary=b64decode(data)
    with open(filename,"wb") as video_file:
      video_file.write(binary)
    print(f"Finished recording video at:{filename}")
  except Exception as err:
    print(str(err))

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 600):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
video_path = "test.mp4"
record_video(video_path)

<IPython.core.display.Javascript object>

Finished recording video at:test.mp4


In [ ]:
show_video(video_path)

In [ ]:
!apt-get install ffmpeg

input_video_path = "/content/test.mp4"
output_audio_path = "/content/output_audio.wav"

!ffmpeg -i "{input_video_path}" -vn -acodec pcm_s16le -ar 44100 -ac 2 "{output_audio_path}"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable

In [ ]:
audio_path = "/content/output_audio.wav"

Audio(audio_path)

In [ ]:
#RAVDESS
!gdown 13NCkIfuEJ-rgEQbqPo4CI31aBUQ2Mc77
!unzip "/content/archive (1).zip" -d "/content/ravdess_dataset/"

# Compare Audio

In [ ]:
path_ = '/content/ravdess_dataset/Actor_16/03-01-02-02-01-01-16.wav'

<h1>Version 1</h1>

In [ ]:
!gdown 1MTGYcfkeXOFPsVq9KnxVRaqlLUhft3lL
!gdown 10beVcAwbYMPyY5xsGpDskebbm6G6jzEJ
!gdown 1EdH1oDM22n8ub8KXQ7qbjynr0nFqfejH

Downloading...
From: https://drive.google.com/uc?id=1MTGYcfkeXOFPsVq9KnxVRaqlLUhft3lL
To: /content/model\emotion_detection_from_audio.h5
100% 6.75M/6.75M [00:00<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?id=10beVcAwbYMPyY5xsGpDskebbm6G6jzEJ
To: /content/scaler.pkl
100% 4.35k/4.35k [00:00<00:00, 23.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EdH1oDM22n8ub8KXQ7qbjynr0nFqfejH
To: /content/encoder.pkl
100% 569/569 [00:00<00:00, 3.84MB/s]


In [ ]:
model_path = '/content/model\emotion_detection_from_audio.h5'
model_1 = load_model(model_path)

In [ ]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def stretch(data):
    return librosa.effects.time_stretch(data,rate=0.8)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sr = 22050 , n_steps = 0.7)

def extract_features(data,sample_rate):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally

    return result

In [ ]:
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

In [ ]:
data_, sample_rate_ = librosa.load(path_)

X_ = np.array(extract_features(data_,sample_rate_))
X_ = scaler.transform(X_.reshape(1,-1))

pred_test_1 = model_1.predict(np.expand_dims(X_, axis=2))
y_pred_1 = encoder.inverse_transform(pred_test_1)
print(y_pred_1[0][0]) #emotion prediction

1/1 [==============================] - 0s 153ms/step
sad


In [ ]:
print(pred_test_1)
print(np.max(pred_test_1))

[[2.1580267e-05 3.3206092e-03 3.9370982e-03 7.0588052e-01 7.0072203e-03
  5.3518757e-02 2.2631149e-01 2.7155456e-06]]
0.7058805


<h2>Version 2</h2>

In [ ]:
!gdown 10trB3r1axMl-P_D8uTK9jAUdXlH0Yi4k
!gdown 1XB2qCo7DSEz3ynalOPEuTYYdY_LoOhvw
!gdown 170qSDaVwTXAxSaC46C_ixZxUaboj1kyz

Downloading...
From: https://drive.google.com/uc?id=10trB3r1axMl-P_D8uTK9jAUdXlH0Yi4k
To: /content/model\emotion_detection_from_merged_audio.h5
100% 6.75M/6.75M [00:00<00:00, 110MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XB2qCo7DSEz3ynalOPEuTYYdY_LoOhvw
To: /content/encoder_merged.pkl
100% 569/569 [00:00<00:00, 3.89MB/s]
Downloading...
From: https://drive.google.com/uc?id=170qSDaVwTXAxSaC46C_ixZxUaboj1kyz
To: /content/scaler_merged.pkl
100% 4.35k/4.35k [00:00<00:00, 21.2MB/s]


In [ ]:
model_path = '/content/model\emotion_detection_from_merged_audio.h5'
model_2 = load_model(model_path)

In [ ]:
with open('scaler_merged.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('encoder_merged.pkl', 'rb') as f:
    encoder = pickle.load(f)

In [ ]:
data_, sample_rate_ = librosa.load(path_)

X_ = np.array(extract_features(data_,sample_rate_))
X_ = scaler.transform(X_.reshape(1,-1))

pred_test_2 = model_2.predict(np.expand_dims(X_, axis=2))
y_pred_2 = encoder.inverse_transform(pred_test_2)
print(y_pred_2[0][0]) #emotion prediction

1/1 [==============================] - 0s 124ms/step
fear


In [ ]:
print(pred_test_2)
print(y_pred_2[0][0])
print(np.max(pred_test_2))

[[1.4666857e-07 2.3613545e-06 1.1697594e-04 1.1317447e-03 1.3236061e-04
  9.0014017e-01 7.2620019e-02 2.5856357e-02]]
0.90014017


<h2>Version 3</h2>

In [ ]:
!gdown 19CjLJ_-QcJCY3LE4Sm-FZPOZwA0qyZFs

Downloading...
From: https://drive.google.com/uc?id=19CjLJ_-QcJCY3LE4Sm-FZPOZwA0qyZFs
To: /content/model\emotion_detection_from_audio_v2.h5
100% 1.41M/1.41M [00:00<00:00, 168MB/s]


In [ ]:
def get_feature(file_name):

        data, sample_rate = librosa.load(file_name)
        stft = np.abs(librosa.stft(data))
        mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40).T, axis=0)
        mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T,axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)



        return mfccs,mel,chroma,contrast

In [ ]:
clf = joblib.load('/content/model\emotion_detection_from_audio_v2.h5')

In [ ]:
feature=[]

mfccs,mel,chroma,contrast = get_feature(path_)
ext_features = np.hstack([mfccs,mel,chroma,contrast])
feature.append(ext_features)

y_pred_3 = clf.predict(feature)
#y_pred_ = encoder.inverse_transform(pred_test_)
print(y_pred_3) #emotion prediction
print(y_pred_3[0]) #emotion prediction

['surprised']
surprised


<h1> Result Comparison </h1>


In [ ]:
path_ = '/content/ravdess_dataset/Actor_12/03-01-04-02-01-02-12.wav'

In [ ]:
print(pred_test_1)
print(y_pred_1[0][0])
print(np.max(pred_test_1))

[[3.9600522e-07 5.3708432e-03 6.5742532e-04 1.1577690e-01 2.8071366e-03
  2.0665544e-01 6.6873169e-01 1.1130942e-07]]
sad
0.6687317


In [ ]:
print(pred_test_2)
print(y_pred_2[0][0])
print(np.max(pred_test_2))

[[6.5450650e-04 3.1654938e-17 6.8410272e-06 9.7038078e-01 9.2135007e-03
  3.7849358e-07 7.5455719e-05 1.9668499e-02]]
fear
0.9703808


In [ ]:
print(y_pred_3)

['surprised']
